[View in Colaboratory](https://colab.research.google.com/github/nenorden/Transfer-learning-on-Fashion-MNIST/blob/master/SVM.ipynb)

# Download Fashion-MNIST

In [1]:
import os
os.chdir('/content/')
!git clone https://github.com/zalandoresearch/fashion-mnist.git 
os.chdir('/content/fashion-mnist/')
import utils.mnist_reader as mnist_reader
os.chdir("/")
train_x_orig, train_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='train')
test_x_orig, test_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='t10k')


fatal: destination path 'fashion-mnist' already exists and is not an empty directory.


# Import ResNet Features

## Download features from Drive to virtual machine

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1j1488T_da1oVlwwy-_19GMuY9tSucqzg'
test_file_id = '1qR7fOsQ6TTW8eV8SnKN-jVvswlzMx9QM'

os.chdir('/content/')
# Load files by ID
downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile('train_fashion-MNIST_resnet.npy')

downloaded = drive.CreateFile({'id': test_file_id})
downloaded.GetContentFile('test_fashion-MNIST_resnet.npy')

# Files should be in /content/. Check!

!ls
os.chdir('/')


datalab        test_fashion-MNIST_resnet.npy
fashion-mnist  train_fashion-MNIST_resnet.npy


## Load ResNet data

In [0]:
%matplotlib inline
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

os.chdir('/content/')
train_x = np.load('train_fashion-MNIST_resnet.npy')
test_x = np.load('test_fashion-MNIST_resnet.npy')
os.chdir('/')


# SVM

## ResNet features

In [4]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import time

# Hyperparameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
svm = svm.LinearSVC() # linear support vector classifier
#svm = GridSearchCV(svm, param_grid)
svm.fit(train_x, train_y)

print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(svm)

# Predict on test data
svm_predict = svm.predict(test_x)

# Get accuracy
svm_acc = (svm_predict == test_y).mean()
print(svm_acc)



--- 1135.4124119281769 seconds ---
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.7702


## Original features

In [9]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
svm_orig = svm.LinearSVC()
svm_orig = GridSearchCV(svm_orig, param_grid)
svm_orig.fit(train_x_orig, train_y)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(svm_orig)

# Predict on test data
svm_predict_orig = svm_orig.predict(test_x_orig)

# Get accuracy
svm_acc_orig = (svm_predict_orig == test_y).mean()
print(svm_acc_orig)


--- 2601.9068768024445 seconds ---
GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1.0], 'max_iter': [500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.7687
